# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [1]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [1]:
import pandas as pd  # pandas for data loading/manipulation :contentReference[oaicite:6]{index=6}
import numpy as np   # numpy for numeric operations :contentReference[oaicite:7]{index=7}
from sklearn.model_selection import train_test_split  # train/test split :contentReference[oaicite:8]{index=8}
from sklearn.preprocessing    import StandardScaler   # standardize features :contentReference[oaicite:9]{index=9}

# 2.1 Load the raw CSV directly from GitHub
spaceship = pd.read_csv(
    "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv"
)  # pandas can read CSV over HTTP(s) :contentReference[oaicite:10]{index=10}

# 2.2 Drop any rows with missing values (low null proportion)
spaceship = spaceship.dropna(axis=0, how="any")  # drop rows containing NaN :contentReference[oaicite:11]{index=11}

# 2.3 Extract deck letter from Cabin (format: Deck/Room/Side)
spaceship["Deck"] = spaceship["Cabin"].str.split("/", expand=True)[0]  # string split on "/" and take index 0 :contentReference[oaicite:12]{index=12}

# 2.4 Drop identifier columns (PassengerId, Name) and original Cabin
spaceship = spaceship.drop(columns=["PassengerId", "Name", "Cabin"])  # no predictive value 

# 2.5 Identify categorical columns excluding target
cat_cols = spaceship.select_dtypes(include=["object", "bool"]).columns.tolist()
cat_cols.remove("Transported")  # retain Transported for y :contentReference[oaicite:14]{index=14}

# 2.6 Perform one-hot encoding on all categorical features (drop_first=True)
spaceship = pd.get_dummies(
    spaceship,
    columns=cat_cols,
    drop_first=True
)  # convert categories to numeric dummy columns :contentReference[oaicite:15]{index=15}

# 2.7 Separate features (X) and target (y)
y = spaceship["Transported"].astype(int)  # convert boolean to 0/1 :contentReference[oaicite:16]{index=16}
X = spaceship.drop(columns=["Transported"])

# 2.8 Train/test split (80% train, 20% test, stratified on y)
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.20,
    random_state=42,
    stratify=y  # maintain class balance :contentReference[oaicite:17]{index=17}
)

# 2.9 Scale numeric features (continuous columns)
numeric_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()  # numeric dtype list :contentReference[oaicite:18]{index=18}

scaler = StandardScaler()
X_train[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])   # fit & transform on train :contentReference[oaicite:19]{index=19}
X_test[numeric_cols]  = scaler.transform(X_test[numeric_cols])       # transform on test :contentReference[oaicite:20]{index=20}

# 2.10 Verify no nulls remain
assert X_train.isnull().sum().sum() == 0
assert X_test.isnull().sum().sum()  == 0

print("Feature Engineering Complete: X_train shape =", X_train.shape, "X_test shape =", X_test.shape)


Feature Engineering Complete: X_train shape = (5284, 19) X_test shape = (1322, 19)


- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [2]:
from sklearn.ensemble import RandomForestClassifier  # RF classifier :contentReference[oaicite:22]{index=22}
from sklearn.metrics   import accuracy_score, classification_report, confusion_matrix  # evaluation metrics :contentReference[oaicite:23]{index=23}

# 3.1 Initialize RandomForestClassifier with default hyperparameters
rf_default = RandomForestClassifier(random_state=42)  # default n_estimators=100, etc. :contentReference[oaicite:24]{index=24}

# 3.2 Train on the training set
rf_default.fit(X_train, y_train)

# 3.3 Predict on the test set
y_pred_default = rf_default.predict(X_test)

# 3.4 Evaluate baseline performance
acc_default = accuracy_score(y_test, y_pred_default)
print("Baseline Random Forest Accuracy:", acc_default)  # expect ~0.81 :contentReference[oaicite:25]{index=25}
print("Classification Report:\n", classification_report(y_test, y_pred_default))  # precision, recall, F1 :contentReference[oaicite:26]{index=26}
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_default))  # TP, TN, FP, FN :contentReference[oaicite:27]{index=27}


Baseline Random Forest Accuracy: 0.7753403933434191
Classification Report:
               precision    recall  f1-score   support

           0       0.77      0.78      0.77       656
           1       0.78      0.77      0.78       666

    accuracy                           0.78      1322
   macro avg       0.78      0.78      0.78      1322
weighted avg       0.78      0.78      0.78      1322

Confusion Matrix:
 [[511 145]
 [152 514]]


- Evaluate your model

In [3]:
# 4.1 Define the hyperparameter grid dictionary
param_grid_rf = {
    "n_estimators": [100, 200, 300],              # number of trees :contentReference[oaicite:30]{index=30}
    "max_depth": [None, 10, 20, 30],               # tree depths :contentReference[oaicite:31]{index=31}
    "max_features": ["sqrt", "log2"],              # features per split :contentReference[oaicite:32]{index=32}
    "min_samples_split": [2, 5, 10],               # samples to split :contentReference[oaicite:33]{index=33}
    "min_samples_leaf": [1, 2, 4],                 # samples at leaf :contentReference[oaicite:34]{index=34}
    "bootstrap": [True, False]                     # bootstrap or not :contentReference[oaicite:35]{index=35}
}


**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

In [4]:
from sklearn.model_selection import GridSearchCV  # exhaustive grid search CV :contentReference[oaicite:37]{index=37}

# 5.1 Initialize the base Random Forest
rf = RandomForestClassifier(random_state=42)

# 5.2 Set up GridSearchCV with 5‐fold CV
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid_rf,
    cv=5,                     # 5-fold cross‐validation :contentReference[oaicite:38]{index=38}
    n_jobs=-1,                # use all cores :contentReference[oaicite:39]{index=39}
    verbose=2,                # print progress messages :contentReference[oaicite:40]{index=40}
    scoring="accuracy"        # optimize for accuracy :contentReference[oaicite:41]{index=41}
)

# 5.3 Fit GridSearch on training data (this will take time)
print("Running GridSearchCV ...")
grid_search.fit(X_train, y_train)

# 5.4 Extract the best hyperparameters and best estimator
best_params = grid_search.best_params_
best_rf     = grid_search.best_estimator_

print("Best Hyperparameters:", best_params)


Running GridSearchCV ...
Fitting 5 folds for each of 432 candidates, totalling 2160 fits
Best Hyperparameters: {'bootstrap': True, 'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 300}


- Run Grid Search

In [5]:
# 6.1 Use the best RF estimator to predict on the test set
y_pred_tuned = best_rf.predict(X_test)

# 6.2 Evaluate tuned model performance
acc_tuned = accuracy_score(y_test, y_pred_tuned)
print("Tuned Random Forest Test Accuracy:", acc_tuned)  # often ~0.82–0.83 :contentReference[oaicite:44]{index=44}

print("Tuned Classification Report:\n", classification_report(y_test, y_pred_tuned))  # improved metrics :contentReference[oaicite:45]{index=45}
print("Tuned Confusion Matrix:\n", confusion_matrix(y_test, y_pred_tuned))  # confusion metrics :contentReference[oaicite:46]{index=46}


Tuned Random Forest Test Accuracy: 0.7987897125567323
Tuned Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.77      0.79       656
           1       0.79      0.82      0.80       666

    accuracy                           0.80      1322
   macro avg       0.80      0.80      0.80      1322
weighted avg       0.80      0.80      0.80      1322

Tuned Confusion Matrix:
 [[507 149]
 [117 549]]


- Evaluate your model

In [7]:
{
  'bootstrap': True,
  'max_depth': 10,
  'max_features': 'sqrt',
  'min_samples_leaf': 1,
  'min_samples_split': 10,
  'n_estimators': 300
}


{'bootstrap': True,
 'max_depth': 10,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 10,
 'n_estimators': 300}